In [ ]:
# q_dict = {'#': ['xmas', 'christmas']}
# tr = TwitterReader('./local_configs/api_keys.json')
# tr.set_query(q_dict)
# tweets = tr.read()['data']
# save_tweets(tweets)

In [ ]:
# reset_db()

In [1]:
from data_access.twitter_reader import TwitterReader
from data_storage import db_manager
from data_wrangling.wrangler import Wrangler
from nltk import pos_tag

In [2]:
wrangler = Wrangler(db_manager.tweet_texts())

In [3]:
db_manager.save_processed_tweets(wrangler.tokenize())

In [4]:
g = db_manager.get_processed_tweets()

In [ ]:
import pandas as pd
import nltk

nltk.download('punkt')

In [ ]:
df = pd.read_csv('./data/tweets.csv')

In [ ]:
df = df['text']
df.to_csv(r'./data/text_only.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

In [ ]:
reader = PlaintextCorpusReader('./data/', '.*\.txt')

In [ ]:
reader.sents('text_only.txt')[1]